# Imports

In [6]:

import pandas as pd
import numpy as np
from datetime import datetime 
import requests
import math
import re

from bs4 import BeautifulSoup

# Collect data

* coleta os 36 prodcts da vitrini

In [7]:
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# url do site
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

# Beautifullsoup
soup = BeautifulSoup( page.text, 'html.parser' )

# ===================  Product Data  =================================
products = soup.find('ul', class_= 'products-listing small')
product_list = products.find_all('article', class_ = 'hm-product-item')

# Product_id
product_list[0].get('data-articlecode')
product_id = [i.get('data-articlecode') for i in product_list]

# Product_category
product_list[0].get('data-category')
product_category = [i.get('data-category') for i in product_list]

# product_name
product_list = products.find_all('a', class_= 'item-link' )
product_list[0].get('title')
product_name = [i.get('title') for i in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [i.get_text() for i in product_list]


data = pd.DataFrame([ product_id, product_category, product_name, product_price ]).T
data.columns = ['product_id','product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S:')
# #plotar
# data.head()

In [8]:
data.shape

(36, 5)

In [9]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1130309005,men_jeans_loose,Baggy Jeans,$ 39.99,2023-10-11 14:00:13:
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 29.99,2023-10-11 14:00:13:
2,0979945001,men_jeans_loose,Loose Jeans,$ 39.99,2023-10-11 14:00:13:
3,1159764002,men_jeans_bootcut,Bootcut Loose Jeans,$ 49.99,2023-10-11 14:00:13:
4,1096385001,men_jeans_loose,Loose Jeans,$ 29.99,2023-10-11 14:00:13:


# Data Collection by Product

* coleta dos prod sxpecifico entra em cada um e faz as rspagem 

In [253]:

# Empty DataFrame
df_compositions = pd.DataFrame()

# cria a lista vazia para colocar as col que nao tenho aqui coletada caso meu novo prod tenha alguma caracteristca que nao peguei no promeiro
aux = []
df_patter = pd.DataFrame(columns= ['Article number','Composition','Fit','color_id','style_id','Size'])

for i in range( len( data ) ):
    # url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    url = 'https://www2.hm.com/en_us/productpage.' + '1130309005' + '.html'
    print(f'product:{url}')

    # Requisita essa url do servidor do site H&M
    page = requests.get( url, headers=headers )

    # Parametro de entrada para o beautifulsoup
    soup = BeautifulSoup( page.text, 'html.parser' )

    # ========================== Color Name ==============================
    product_list = soup.find_all('a', class_='filter-option miniature') 
    color_name = [i.get('data-color') for i in product_list]

    # ========================== product_id ==============================
    product_id = [i.get('data-articlecode') for i in product_list]
    
    # ========================== Ccriando o data frame ====================
    df_color = pd.DataFrame([product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']

    for c in range( len( df_color )  ):
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[c, 'product_id'] + '.html'
        print('Color: {}'.format(url))        
    
        page = requests.get( url, headers=headers )
        # Parametro de entrada para o beautifulsoup
        soup = BeautifulSoup( page.text, 'html.parser' )

        # ==============================  Product  ===========================================
        product_element_list = soup.find_all('section', class_='product-name-price')
        
        all_product_info_color = []
        
        for product_element in product_element_list:
            product_info_color = {}
            
            # =====================  product_name =================================================
            color_element = product_element.find('h1', string= 'Baggy Jeans')    
            if color_element:
                color = re.search(r'productName = \'(.*?)\';', color_element.find_next('script').text.strip()).group(1)
                product_info_color['Color_Name'] = color
                
            # =============================== Product Price  ========================================
            price_tag = soup.find('span', class_='price-value')
            if price_tag:
                price = price_tag.get_text(strip=True)
                product_info_color['Price'] = price
                
            # Adicionar as informações do produto à lista
            all_product_info_color.append(product_info_color)
            
            # criar data frame 
            df_composition = pd.DataFrame(all_product_info_color)    
            
            # =============================== Compositiom =======================================
        
            # Encontrei todos os elementos que representam os produtos
            product_elements_list = soup.find_all('div', class_='content pdp-text pdp-content')
        
            # Inicializei uma lista para armazenar as informações de todos os produtos
            all_products_info = []
            
            for product_element in product_elements_list:
                product_info = {}
    
                # FIT
                fit_element = product_element.find('dt', string='Fit')
                if fit_element:
                    fit = fit_element.find_next('dd').text.strip()
                    product_info['Fit'] = fit
    
                # SIZE
                size_element = product_element.find('dt', string='Size')
                if size_element:
                    size = size_element.find_next('dd').text.strip()
                    product_info['Size'] = size    
    
                # COMPOSITION
                composition_element = product_element.find('h3', string='Composition')
                if composition_element:
                    composition = composition_element.find_next('ul').text.strip()
                    composition_cleaned = re.sub(r'(Pocket|lining|Shell)', '', composition, flags=re.IGNORECASE)
                    product_info['Composition'] = composition_cleaned.replace('\n', '')  
                
                # ARTICLE NUMBER
                article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
                if article_number_element:
                    article_number_text = article_number_element.text.strip()
                    article_number = re.search( r'\d+', article_number_text ).group()
                    product_info['Article_number'] =  article_number  
    
                # Adicionar as informações do produto à lista
                all_products_info.append(product_info)
    
                # criar data frame 
                df_composition = pd.DataFrame(all_products_info)
                
                # Garante the same number o columns   (Padrao de dados que eu espero receber) 
                df_composition = pd.concat([ df_patter, df_composition], axis=0)
                
                # rename columns
                df_composition.columns = ['Article_number', 'Composition', 'Fit', 'Size', 'product_id']
                
                # guardo aqui as col  que nao foi prevista 
                aux = aux + df_composition.columns.tolist()
                
                # merge color + composition  
                df_composition = pd.merge( df_composition, df_color, how='left', on='product_id')
                
                print(df_composition)
                
                # All product
                df_compositions = pd.concat( [df_compositions, df_composition], axis=0)
                
                
                
hkkhkhkhkhkhkh   
            
            
                       
        #  generate style id+ color id
        #  df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
        #  df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    #    
        #  df_composition['style_id'] = df_composition['Article number'].apply( lambda x: x[:-3])
#     df_composition['color_id'] = df_composition['Article number'].apply( lambda x: x[-3:])
    

#     # ========================== df_color + df_composition ================================== 
#     if 'Size' in df_composition.columns:
#        data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Size']], how='left', on='style_id')
#     else:
#        data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

    
#     #data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit', 'Composition','Pocket lining', 'Size']], how='left', on='style_id')
    
#     df_final = pd.concat([df_final, data_sku], axis=0)
    
# # Join showroom data + df_final
    
# data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
# data['color_id'] = data['product_id'].apply( lambda x: x[-3:])

# data_raw = pd.merge(data, df_final[['style_id','color_name', 'Fit', 'Composition', 'Size']], how='left', on='style_id')    

product:https://www2.hm.com/en_us/productpage.1130309005.html
Color: https://www2.hm.com/en_us/productpage.1130309001.html


ValueError: Length mismatch: Expected axis has 7 elements, new values have 5 elements

In [252]:
df_compositions

,Article_number,Composition,Fit,Color_ID,style_id,Size,product_id,color_name
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,NaN,1130309001,Denim black
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,NaN,1130309002,Light denim blue
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 31/32",1130309004,Dark denim gray
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 31/32",1130309007,Dark denim gray
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,NaN,1130309008,Denim red
...,...,...,...,...,...,...,...,...
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,NaN,1130309002,Light denim blue
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 31/32",1130309004,Dark denim gray
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 31/32",1130309007,Dark denim gray
0,NaN,"Cotton 100% Polyester 65%, Cotton 35%",Loose fit,NaN,NaN,NaN,1130309008,Denim red


In [217]:
product_picee = soup.find_all( 'div',  class_="primary-row product-item-price" )
product_picee = re.findall(r'\d+\.?\d+', product_picee[0].get_text())[0]git add .
git commit -m ''
product_picee

'39.99'

# Data Cleaning

* limpeza e separacao dados:

In [ ]:
# data = pd.read_csv('./dataset/data_web.csv')
# # renomeando as cols
# data.rename( columns= lambda x: x.lower(), inplace=True)


# # ================================== transfommacao das col ======================================

# data = data.dropna(subset=['product_id'])
# data['product_id'] = data['product_id'].astype(int)

# # product_name
# data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# # product_price - Remover o símbolo '$' e converter para float
# data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '') if isinstance(x, str) else x ).astype(float)

# # scrapy_datetime
# data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S:')

# # Color Name
# data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if isinstance(x, str) else x)

# # Fit
# data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# # pocket lining  / Removendo a coluna 'poket lining'

# # Size
# data['size'] = data['size'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

# # size_number - Extraido números da coluna 'size'
# data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if isinstance(x, str) and re.search('\d{3}cm', x) else x)
# data['size_number'] = data['size_number'].apply( lambda x: re.search('\d+', x).group(0) if pd.notnull( x ) else x )

# # size_model - Extraido números da coluna 'size'
# data['size_model'] = data['size'].apply( lambda x: re.search('\d+/\\d+', x).group(0) if isinstance(x, str) and re.search('\d+/\\d+', x) else x  )

# # ========================== Drop duplicates ==================================================
# data = data.drop_duplicates(subset=['product_id', 'product_name', 'product_category', 'product_price','scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last')

# # reset Index
# data = data.reset_index(drop=True)

# # ========================== brack composition by comma =================================

# df1 = data['composition'].str.split(',', expand=True)

# #criando um dataframe vazio do tamanho de data para alocar as colunas em ordem
# df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton', 'Spandex', 'Elastomultiester'] )
 
# # ======= DF Cotton =====================

# df_cotton = df1[0] # data que tem só cotton
# df_cotton.name = 'cotton' # passo o name para a coluna
 
# df_ref = pd.concat([ df_ref, df_cotton], axis=1) # faco a uniao com concat 
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')] # e dorp a col original q era so NaN

# df_ref['cotton'] = df_ref['cotton'].fillna('cotton 0%') # td coton vzio passsa a ser coton 0%

# # ============  DF Spandex  =======================

# df_spandex = df1.loc[df1[1].str.contains('spandex', na=False), 1]
# df_spandex.name = 'spandex'

# # combine spandex from brch colum 1 and 2
# df_spandex =df_spandex.combine_first( df1[2] )

# df_ref = pd.concat([df_ref, df_spandex], axis=1)
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
# df_ref['spandex'] = df_ref['spandex'].fillna('spandex 0%')

# # ================  DF Elastomultiester  ===========================
	
# df_Elastomultiester	 = df1.loc[ df1[1].str.contains('Elastomultiester', na=False), 1]
# df_Elastomultiester.name = 'Elastomultiester'

# df_ref = pd.concat([ df_ref, df_Elastomultiester], axis=1)
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
# df_ref['Elastomultiester'] = df_ref['Elastomultiester'].fillna('Elastomultiester 0%')

# df_ref = df_ref.drop('Spandex', axis=1)

# # ========= Concat ================

# data = pd.concat([data, df_ref], axis=1)


# # =====================  format composition data =====================================

# # cotton
# data['cotton'] = data['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )

# #spandex.
# data['spandex'] = data['spandex'].apply( lambda x: int( re.search('\d+', x).group(0) ) /100 if pd.notnull(x) else x )

# # Elastomultiester
# data['Elastomultiester'] = data['Elastomultiester'].apply( lambda x: int(re.search('\d+', x).group(0))/ 100 if pd.notnull( x ) else x)




# # ============ Check =================================

# data = data.drop( columns=['pocket lining', 'size', 'composition'], axis=1 )
# data = data.drop_duplicates()
# data.shape


# # criando um csv
# # dados = 'data_webs_treated.csv'
# # data.to_csv(dados, index=False)